In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils import shuffle
from scipy.stats import norm
import json

sns.set(color_codes=True)

In [2]:
import_file_name = './scrapped_db.buildings.csv'
export_file_name = './ny_seattle.csv'

json_list = []
with open('./nySeattle.json') as f:   
    for n, line in enumerate(f):
        try:
            json_list.append(json.loads(line))
        except:
            print(n)

df = pd.DataFrame(json_list)
df.head()

,_id,addr:housenumber,addr:street,air_quality,amenity,area,bbox_id,building,building_centroid,closest_shop,...,description,geometry,height,humidity,name,request_area_bottom_left,request_area_top_right,shops_amount,shops_count,temperature
0,{'$oid': '5bd44a65c6b61430207da0ce'},199,Water Street,80,NaN,2929.080229,NY1,None,"{'type': 'Point', 'coordinates': [-74.00436560...",48,...,NaN,"{'type': 'Polygon', 'coordinates': [[[-74.0045...",15.0,0.71,One Seaport Plaza,"{'type': 'Point', 'coordinates': [-74.015751, ...","{'type': 'Point', 'coordinates': [-74.004765, ...",77.0,NaN,12.32
1,{'$oid': '5bd44a65c6b61430207da0cf'},175,Water Street,80,NaN,1755.001543,NY1,None,"{'type': 'Point', 'coordinates': [-74.00515135...",33,...,NaN,"{'type': 'Polygon', 'coordinates': [[[-74.0050...",15.0,0.71,NaN,"{'type': 'Point', 'coordinates': [-74.015751, ...","{'type': 'Point', 'coordinates': [-74.004765, ...",85.0,NaN,12.32
2,{'$oid': '5bd44a65c6b61430207da0d0'},NaN,NaN,80,NaN,2547.115851,NY1,None,"{'type': 'Point', 'coordinates': [-74.00475392...",50,...,NaN,"{'type': 'Polygon', 'coordinates': [[[-74.0052...",30.0,0.71,200 Water Street,"{'type': 'Point', 'coordinates': [-74.015751, ...","{'type': 'Point', 'coordinates': [-74.004765, ...",81.0,NaN,12.32
3,{'$oid': '5bd44a65c6b61430207da0d1'},99,Washington Street,80,NaN,510.346791,NY1,None,"{'type': 'Point', 'coordinates': [-74.01411221...",22,...,NaN,"{'type': 'Polygon', 'coordinates': [[[-74.0142...",15.0,0.71,99 Washington Street,"{'type': 'Point', 'coordinates': [-74.015751, ...","{'type': 'Point', 'coordinates': [-74.004765, ...",52.0,NaN,12.32
4,{'$oid': '5bd44a65c6b61430207da0d2'},NaN,NaN,88,NaN,249.167397,NY1,None,"{'type': 'Point', 'coordinates': [-74.01533099...",147,...,NaN,"{'type': 'Polygon', 'coordinates': [[[-74.0154...",0.0,0.71,NaN,"{'type': 'Point', 'coordinates': [-74.015751, ...","{'type': 'Point', 'coordinates': [-74.004765, ...",19.0,NaN,12.32


In [3]:
df.describe()

,air_quality,area,closest_shop,cloud_cover,height,humidity,shops_amount,shops_count,temperature
count,11477.000000,11477.000000,11477.000000,11477.000000,11477.000000,11477.000000,10630.000000,847.0,11477.000000
mean,83.212076,348.131706,318.170863,0.585594,8.905115,0.747690,15.319755,0.0,11.779452
std,5.658455,860.814529,370.720630,0.014814,11.848408,0.033447,21.645164,0.0,0.497374
min,58.000000,7.582047,0.000000,0.570000,0.000000,0.700000,0.000000,0.0,10.720000
25%,80.000000,100.748974,59.000000,0.570000,0.000000,0.720000,1.000000,0.0,11.310000
50%,84.000000,144.033496,155.000000,0.600000,15.000000,0.760000,8.000000,0.0,11.640000
75%,88.000000,246.946880,455.000000,0.600000,15.000000,0.780000,23.000000,0.0,12.250000
max,89.000000,28475.412404,1682.000000,0.600000,320.000000,0.810000,173.000000,0.0,12.770000


In [4]:
df['bbox_id'].unique()

array(['NY1', 'NY2', 'NY3', 'NY4', 'NY5', 'NY6', 'NY7', 'NY8', 'NY9',
       'NY10', 'Seattle1', 'Seattle2', 'Seattle3', 'Seattle4', 'Seattle5',
       'Seattle6', 'Seattle7', 'Seattle8', 'Seattle9', 'Seattle10'],
      dtype=object)

In [5]:
kiev_df = df.loc[df["bbox_id"].str.startswith("NY")]
lviv_df = df.loc[df["bbox_id"].str.startswith("Seattle")]

In [6]:
print("Number of buildings in Kiev: {}".format(kiev_df['bbox_id'].count()))
print("Number of buildings in Lviv: {}".format(lviv_df['bbox_id'].count()))

Number of buildings in Kiev: 5611
Number of buildings in Lviv: 5866


In [7]:
# select random data from dataframe

result_df = pd.DataFrame()

limit = 100

dfs = [kiev_df, lviv_df]
for df in dfs:
    ids = df['bbox_id'].unique()
    for curr_id in ids:
        part = df[df['bbox_id'] == curr_id].copy()
        result_df = result_df.append(shuffle(part).head(limit))
        
result_df.head()

,_id,addr:housenumber,addr:street,air_quality,amenity,area,bbox_id,building,building_centroid,closest_shop,...,description,geometry,height,humidity,name,request_area_bottom_left,request_area_top_right,shops_amount,shops_count,temperature
298,{'$oid': '5bd44a65c6b61430207da1f8'},195,Broadway,73,NaN,3423.975940,NY1,None,"{'type': 'Point', 'coordinates': [-74.00978367...",33,...,NaN,"{'type': 'Polygon', 'coordinates': [[[-74.0102...",114.0,0.71,195 Broadway,"{'type': 'Point', 'coordinates': [-74.015751, ...","{'type': 'Point', 'coordinates': [-74.004765, ...",82.0,NaN,12.30
160,{'$oid': '5bd44a65c6b61430207da16e'},NaN,NaN,73,NaN,126.140120,NY1,None,"{'type': 'Point', 'coordinates': [-74.01006205...",76,...,NaN,"{'type': 'Polygon', 'coordinates': [[[-74.0101...",0.0,0.71,NaN,"{'type': 'Point', 'coordinates': [-74.015751, ...","{'type': 'Point', 'coordinates': [-74.004765, ...",38.0,NaN,12.30
104,{'$oid': '5bd44a65c6b61430207da136'},94,Reade Street,73,NaN,129.614227,NY1,None,"{'type': 'Point', 'coordinates': [-74.00784519...",19,...,NaN,"{'type': 'Polygon', 'coordinates': [[[-74.0078...",0.0,0.71,NaN,"{'type': 'Point', 'coordinates': [-74.015751, ...","{'type': 'Point', 'coordinates': [-74.004765, ...",37.0,NaN,12.30
337,{'$oid': '5bd44a65c6b61430207da21f'},30,Broad Street,80,NaN,1126.547544,NY1,None,"{'type': 'Point', 'coordinates': [-74.01165412...",15,...,NaN,"{'type': 'Polygon', 'coordinates': [[[-74.0119...",15.0,0.71,Continental Bank Building,"{'type': 'Point', 'coordinates': [-74.015751, ...","{'type': 'Point', 'coordinates': [-74.004765, ...",109.0,NaN,12.32
557,{'$oid': '5bd44a65c6b61430207da2fb'},19,Murray Street,73,NaN,220.214769,NY1,None,"{'type': 'Point', 'coordinates': [-74.0081135,...",35,...,NaN,"{'type': 'Polygon', 'coordinates': [[[-74.0081...",0.0,0.71,NaN,"{'type': 'Point', 'coordinates': [-74.015751, ...","{'type': 'Point', 'coordinates': [-74.004765, ...",54.0,NaN,12.30


In [8]:
# check that data are uniformly destributed by `bbox_id`
for bbox_id in result_df['bbox_id'].unique():
    print('Count of buildings in {}: {}'.format(bbox_id, result_df[result_df['bbox_id'] == bbox_id]['_id'].count()))

Count of buildings in NY1: 100
Count of buildings in NY2: 100
Count of buildings in NY3: 100
Count of buildings in NY4: 100
Count of buildings in NY5: 100
Count of buildings in NY6: 100
Count of buildings in NY7: 100
Count of buildings in NY8: 100
Count of buildings in NY9: 100
Count of buildings in NY10: 100
Count of buildings in Seattle1: 100
Count of buildings in Seattle2: 100
Count of buildings in Seattle3: 100
Count of buildings in Seattle4: 100
Count of buildings in Seattle5: 100
Count of buildings in Seattle6: 100
Count of buildings in Seattle7: 100
Count of buildings in Seattle8: 100
Count of buildings in Seattle9: 100
Count of buildings in Seattle10: 100


In [9]:
result_df.to_csv(export_file_name)